# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [124]:
import csv
import json
import os
import requests
import this
from typing import final

import pandas as pd
from pandas.core.interchange.dataframe_protocol import DataFrame

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [106]:
def read_json(path: str):
    with open(path) as json_file:
        json_data = json.load(json_file)
        project_artifact = json_data["projectArtifact"]
        file_import_reports = json_data["fileImportReports"]
        project_unused_dependencies = json_data["projectUnusedDependencies"]
        project_use_dependencies = json_data["projectUseDependencies"]
        return project_artifact, file_import_reports, project_use_dependencies, project_unused_dependencies

def list_json_files(directory):
    try:
        return [f for f in os.listdir(directory) if f.endswith('.json')]
    except FileNotFoundError:
        print(f"Error: Directory '{directory}' not found.")
        return []
    except Exception as e:
        print(f"An error occurred: {e}")
        return []
    
def calculate_import_but_unused_dependencies(project_unused_dependencies, file_import_reports):
    project_unused_dependencies_list = []
    result = []
    for unused_dependency in project_unused_dependencies:
        artifact = unused_dependency["artifact"]
        project_unused_dependencies_list.append(artifact)
    
    for file in file_import_reports:
        unused_import_report = file["unusedImportReport"]
        for unused_import in unused_import_report:
            artifact = unused_import["artifact"]["artifact"]
            if (artifact in project_unused_dependencies_list):
                result.append(artifact)
    return len(set(result))

def calculate_file_level_unused_import(file_import_reports):
    total_use_import = 0
    total_unused_import = 0
    num_files = 0
    for file_report in file_import_reports:
        num_use_import_report = len(file_report["useImportReport"])
        num_unused_import_report = len(file_report["unusedImportReport"])
        total_use_import += num_use_import_report
        total_unused_import += num_unused_import_report
        num_files += 1
    return total_use_import, total_unused_import, num_files
    
# read_json("data/new-dependency-output/at.molindo.social:spring-social-openid:1.1.1.RELEASE.json")

In [79]:
class DependenciesDetails:
    group_df = None
    df = None
    
    def __init__(self):
        csv_file = "data/artifact-dependency-details.csv"
        self.df = pd.read_csv(csv_file)
        
    def get_artifact_details(self, release_id):
        # Group by 'release_id' and get the first row of each group
        return self.df[self.df['release_id'] == release_id]

        

In [118]:
json_file_list = list_json_files("data/new-dependency-output")
dependency_details = DependenciesDetails()

list_df = []
for file in json_file_list:
    project_artifact, file_import_reports, project_use_dependencies, project_unused_dependencies = read_json("data/new-dependency-output/" + file)
    row = dependency_details.get_artifact_details(project_artifact).copy()
    import_but_unused = calculate_import_but_unused_dependencies(project_unused_dependencies, file_import_reports)
    num_using_import_in_file, num_unused_import_in_file, num_files = calculate_file_level_unused_import(file_import_reports)
    row['num_use_import'] = len(project_use_dependencies)
    row['num_unused_import'] = len(project_unused_dependencies)
    row['num_import_but_unused'] = import_but_unused
    row['num_use_import_in_file'] = num_using_import_in_file
    row['num_unused_import_in_file'] = num_unused_import_in_file
    row['avg_use_import_in_each_file'] = num_using_import_in_file / num_files
    row['avg_unused_import_in_each_file'] = num_unused_import_in_file / num_files
    row['num_files'] = num_files
    list_df.append(row)
    
final_df = pd.concat(list_df, ignore_index=True)
final_df

,artifact_id,release_id,version,timestamp,popularity,all_dependency,compile_dependency,provided_dependency,dependency,num_use_import,num_unused_import,num_import_but_unused,num_use_import_in_file,num_unused_import_in_file,avg_use_import_in_each_file,avg_unused_import_in_each_file,num_files
0,com.fuzhutech:fuzhutech-ueditor,com.fuzhutech:fuzhutech-ueditor:1.0.0,1.0.0,1495562978000,0,7,6,0,"[""org.json:json:20160810:compile"",""javax.servl...",5,1,0,134,6,5.826087,0.260870,23
1,org.eurekaclinical:i2b2-client,org.eurekaclinical:i2b2-client:2.3,2.3,1597331951000,0,6,6,0,"[""javax.xml.bind:jaxb-api:2.3.1:compile"",""com....",3,3,0,167,0,4.513514,0.000000,37
2,org.jeecgframework.nacos:nacos-api,org.jeecgframework.nacos:nacos-api:2.3.2,2.3.2,1720620475000,7,8,8,0,"[""io.grpc:grpc-stub:1.57.2:compile"",""io.grpc:g...",7,3,0,347,9,2.090361,0.054217,166
3,org.codelibs:elasticsearch-analysis-kuromoji-u...,org.codelibs:elasticsearch-analysis-kuromoji-u...,6.6.0,1550711556000,0,5,2,1,"[""org.codelibs:lucene-analyzers-kuromoji-unidi...",3,0,0,96,0,9.600000,0.000000,10
4,uk.uuid.slf4j:slf4j-android,uk.uuid.slf4j:slf4j-android:2.0.16-0,2.0.16-0,1723635390000,0,8,1,1,"[""com.google.code.findbugs:annotations:3.0.1u2...",2,0,0,27,1,2.700000,0.100000,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,org.codelibs.fess:fess-ds-salesforce,org.codelibs.fess:fess-ds-salesforce:14.10.0,14.10.0,1691309393000,0,12,6,4,"[""com.force.api:force-wsc:47.0.0:compile"",""org...",5,5,0,180,0,5.625000,0.000000,32
96,org.apache.sling:feature-launcher-maven-plugin,org.apache.sling:feature-launcher-maven-plugin...,0.1.6,1688727219000,0,9,4,4,"[""org.apache.maven:maven-artifact:3.3.9:provid...",8,0,0,65,0,13.000000,0.000000,5
97,com.github.davidmarquis:redis-scheduler,com.github.davidmarquis:redis-scheduler:3.0.0,3.0.0,1523196258000,0,12,1,6,"[""redis.clients:jedis:2.9.0:provided"",""org.spr...",4,3,0,41,0,3.727273,0.000000,11
98,com.aliyun:alibabacloud-aligenieip_1_0,com.aliyun:alibabacloud-aligenieip_1_0:1.0.1,1.0.1,1709196418000,0,3,1,0,"[""com.aliyun:aliyun-gateway-pop:0.2.7-beta:com...",1,0,0,1382,355,4.749141,1.219931,291


In [123]:
summary_df = final_df[['num_use_import', 'num_unused_import', 'num_import_but_unused', 'avg_use_import_in_each_file', 'avg_unused_import_in_each_file', 'num_files']]

summary = pd.DataFrame({
    'mean': summary_df.mean(),
    'median': summary_df.median(),
    'max': summary_df.max(),
    'min': summary_df.min(),
    'std': summary_df.std()  # Standard deviation
})

# Rename index to show column names from the original DataFrame
summary.index.name = 'Column'

# Display the summary table
summary

,mean,median,max,min,std
Column,,,,,
num_use_import,4.880000,4.000000,31.000000,0.0,5.224805
num_unused_import,1.800000,1.000000,25.000000,0.0,3.149475
num_import_but_unused,0.010000,0.000000,1.000000,0.0,0.100000
avg_use_import_in_each_file,7.997980,6.765067,30.428571,0.0,5.100419
avg_unused_import_in_each_file,0.125827,0.000000,3.000000,0.0,0.390227
num_files,404.290000,15.000000,33041.000000,1.0,3299.632218
